In [1]:
from sqlalchemy.orm import declarative_base

# Model

In [2]:
import os
from pathlib import Path
#BASE_DIR = os.path.dirname(os.path.realpath(__file__))
BASE_DIR = Path().resolve() #esto funciona en notebook
connection_string = "sqlite:///" + os.path.join(BASE_DIR, 'site.db')
connection_string = "sqlite:///:memory:"
print (connection_string)

sqlite:///:memory:


In [3]:
from sqlalchemy import Column, Integer, String, DateTime, create_engine
from datetime import datetime

Base = declarative_base()

engine = create_engine(connection_string, echo=True)
"""
class User
    id int
    username str
    email str
    date_create datetime
"""

class User(Base):
    __tablename__ = "users"
    id = Column(Integer, primary_key=True)
    username = Column(String(25), nullable=False, unique=True)
    email = Column(String(80), nullable=False, unique=True)
    date_created = Column(DateTime, default=datetime.utcnow())
    
    def __repr__ (self):
        return f"<User username={self.username} email={self.email}>"
    
new_user = User(id=1, username="jonathan", email = "jone@hui.com")
print (new_user)

<User username=jonathan email=jone@hui.com>


In [4]:
#create database if not exists
Base.metadata.create_all(engine)

2022-11-28 14:27:54,349 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-28 14:27:54,350 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2022-11-28 14:27:54,350 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-11-28 14:27:54,351 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("users")
2022-11-28 14:27:54,351 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-11-28 14:27:54,352 INFO sqlalchemy.engine.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	username VARCHAR(25) NOT NULL, 
	email VARCHAR(80) NOT NULL, 
	date_created DATETIME, 
	PRIMARY KEY (id), 
	UNIQUE (username), 
	UNIQUE (email)
)


2022-11-28 14:27:54,352 INFO sqlalchemy.engine.Engine [no key 0.00025s] ()
2022-11-28 14:27:54,353 INFO sqlalchemy.engine.Engine COMMIT


session sirve para hacer las transacciones (lazy¿?)

In [5]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker()
local_session = Session(bind=engine)

# Insert

In [6]:
new_user = User(username="frank", email = "frank@sinatra.com")
local_session.add(new_user)
local_session.commit()

2022-11-28 14:27:54,397 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-28 14:27:54,398 INFO sqlalchemy.engine.Engine INSERT INTO users (username, email, date_created) VALUES (?, ?, ?)
2022-11-28 14:27:54,399 INFO sqlalchemy.engine.Engine [generated in 0.00112s] ('frank', 'frank@sinatra.com', '2022-11-28 17:27:54.304907')
2022-11-28 14:27:54,401 INFO sqlalchemy.engine.Engine COMMIT


In [7]:
users = [
    {"username":"jhonny",
    "email":"jh@company.com"},
    {"username":"jerry",
    "email":"je@company.com"},
    {"username":"lucas",
    "email":"lu@company.com"},
    {"username":"igor",
    "email":"ig@company.com"},
    {"username":"gaby",
    "email":"gb@company.com"},
    {"username":"tom",
    "email":"tm@company.com"},
]

In [8]:
for u in users:
    new_user = User(username = u["username"], email = u["email"])
    local_session.add(new_user)    
    local_session.commit()

2022-11-28 14:27:54,463 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-28 14:27:54,464 INFO sqlalchemy.engine.Engine INSERT INTO users (username, email, date_created) VALUES (?, ?, ?)
2022-11-28 14:27:54,464 INFO sqlalchemy.engine.Engine [cached since 0.06574s ago] ('jhonny', 'jh@company.com', '2022-11-28 17:27:54.304907')
2022-11-28 14:27:54,465 INFO sqlalchemy.engine.Engine COMMIT
2022-11-28 14:27:54,466 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-28 14:27:54,466 INFO sqlalchemy.engine.Engine INSERT INTO users (username, email, date_created) VALUES (?, ?, ?)
2022-11-28 14:27:54,466 INFO sqlalchemy.engine.Engine [cached since 0.06785s ago] ('jerry', 'je@company.com', '2022-11-28 17:27:54.304907')
2022-11-28 14:27:54,467 INFO sqlalchemy.engine.Engine COMMIT
2022-11-28 14:27:54,467 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-28 14:27:54,468 INFO sqlalchemy.engine.Engine INSERT INTO users (username, email, date_created) VALUES (?, ?, ?)
2022-11-28 14:27:54,4

# Query

In [9]:
users = local_session.query(User).all()
for user in users:
    print (user.username)

2022-11-28 14:27:54,530 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-28 14:27:54,536 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.username AS users_username, users.email AS users_email, users.date_created AS users_date_created 
FROM users
2022-11-28 14:27:54,538 INFO sqlalchemy.engine.Engine [generated in 0.00223s] ()
frank
jhonny
jerry
lucas
igor
gaby
tom


In [10]:
users = local_session.query(User).all()[:3]
for user in users:
    print (user.username)

2022-11-28 14:27:54,593 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.username AS users_username, users.email AS users_email, users.date_created AS users_date_created 
FROM users
2022-11-28 14:27:54,594 INFO sqlalchemy.engine.Engine [cached since 0.05798s ago] ()
frank
jhonny
jerry


# Update

In [11]:
#gaby
user = local_session.query(User).filter(User.username == "gaby").first()


2022-11-28 14:27:54,630 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.username AS users_username, users.email AS users_email, users.date_created AS users_date_created 
FROM users 
WHERE users.username = ?
 LIMIT ? OFFSET ?
2022-11-28 14:27:54,630 INFO sqlalchemy.engine.Engine [generated in 0.00077s] ('gaby', 1, 0)


In [12]:
print (user)

<User username=gaby email=gb@company.com>


In [13]:
#updating

In [14]:
user

<User username=gaby email=gb@company.com>

In [15]:
user.username="gabriela"
user.email = "gabriela@company.com"

In [16]:
local_session.commit()

2022-11-28 14:27:54,821 INFO sqlalchemy.engine.Engine UPDATE users SET username=?, email=? WHERE users.id = ?
2022-11-28 14:27:54,822 INFO sqlalchemy.engine.Engine [generated in 0.00067s] ('gabriela', 'gabriela@company.com', 6)
2022-11-28 14:27:54,823 INFO sqlalchemy.engine.Engine COMMIT


# Delete

In [17]:
user_to_delete = local_session.query(User).filter(User.username == "gabriela").first()

2022-11-28 14:27:54,847 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-28 14:27:54,848 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.username AS users_username, users.email AS users_email, users.date_created AS users_date_created 
FROM users 
WHERE users.username = ?
 LIMIT ? OFFSET ?
2022-11-28 14:27:54,848 INFO sqlalchemy.engine.Engine [cached since 0.2183s ago] ('gabriela', 1, 0)


In [18]:
local_session.delete(user_to_delete)

In [19]:
local_session.commit()

2022-11-28 14:27:54,924 INFO sqlalchemy.engine.Engine DELETE FROM users WHERE users.id = ?
2022-11-28 14:27:54,925 INFO sqlalchemy.engine.Engine [generated in 0.00061s] (6,)
2022-11-28 14:27:54,925 INFO sqlalchemy.engine.Engine COMMIT


# Order

In [20]:
users = local_session.query(User).order_by(User.username).all()

2022-11-28 14:27:54,964 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-28 14:27:54,965 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.username AS users_username, users.email AS users_email, users.date_created AS users_date_created 
FROM users ORDER BY users.username
2022-11-28 14:27:54,965 INFO sqlalchemy.engine.Engine [generated in 0.00040s] ()


In [21]:
users

[<User username=frank email=frank@sinatra.com>,
 <User username=igor email=ig@company.com>,
 <User username=jerry email=je@company.com>,
 <User username=jhonny email=jh@company.com>,
 <User username=lucas email=lu@company.com>,
 <User username=tom email=tm@company.com>]

In [22]:
from sqlalchemy import desc
users = local_session.query(User).order_by(desc(User.username)).all()
users

2022-11-28 14:27:55,030 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.username AS users_username, users.email AS users_email, users.date_created AS users_date_created 
FROM users ORDER BY users.username DESC
2022-11-28 14:27:55,031 INFO sqlalchemy.engine.Engine [generated in 0.00067s] ()


[<User username=tom email=tm@company.com>,
 <User username=lucas email=lu@company.com>,
 <User username=jhonny email=jh@company.com>,
 <User username=jerry email=je@company.com>,
 <User username=igor email=ig@company.com>,
 <User username=frank email=frank@sinatra.com>]

In [23]:
import sqlalchemy
sqlalchemy.__version__

'1.4.43'

In [24]:
%reset -f

Otro Ejemplo

In [25]:
from sqlalchemy import create_engine, ForeignKey, Column, String, Integer, CHAR
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

Base = declarative_base()

class Persona(Base):
    __tablename__ = "personas"
    
    ssn = Column(Integer, primary_key=True)
    firstname = Column(String)
    lastname = Column(String)
    gender = Column(CHAR)
    age = Column(Integer)
    
    def __init__(self, ssn, firstname, lastname, gender, age):
        self.ssn = ssn
        self.firstname = firstname
        self.lastname = lastname
        self.gender = gender.upper()
        self.age = age
        
    def __repr__(self):
        return f"{self.ssn} {self.firstname} {self.lastname} ({self.gender}, {self.age})"

In [26]:
engine = create_engine("sqlite:///:memory:", echo=True)

In [27]:
Base.metadata.create_all(bind=engine)

2022-11-28 14:27:55,270 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-28 14:27:55,271 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("personas")
2022-11-28 14:27:55,271 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-11-28 14:27:55,272 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("personas")
2022-11-28 14:27:55,272 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-11-28 14:27:55,273 INFO sqlalchemy.engine.Engine 
CREATE TABLE personas (
	ssn INTEGER NOT NULL, 
	firstname VARCHAR, 
	lastname VARCHAR, 
	gender CHAR, 
	age INTEGER, 
	PRIMARY KEY (ssn)
)


2022-11-28 14:27:55,273 INFO sqlalchemy.engine.Engine [no key 0.00063s] ()
2022-11-28 14:27:55,274 INFO sqlalchemy.engine.Engine COMMIT


In [28]:
Session = sessionmaker(bind=engine)
session = Session()

In [29]:
tom = Persona(21878, "Mike", "Smith", "m", 35)
session.add(tom)
session.commit()

2022-11-28 14:27:55,332 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-28 14:27:55,334 INFO sqlalchemy.engine.Engine INSERT INTO personas (ssn, firstname, lastname, gender, age) VALUES (?, ?, ?, ?, ?)
2022-11-28 14:27:55,334 INFO sqlalchemy.engine.Engine [generated in 0.00062s] (21878, 'Mike', 'Smith', 'M', 35)
2022-11-28 14:27:55,335 INFO sqlalchemy.engine.Engine COMMIT


In [30]:
p1 = Persona(8219821, "Jhon", "Smith", "m", 31)
p2 = Persona(209130938, "Lucas", "Row", "m", 23)
p3 = Persona(99999, "Claudia", "López", "f", 45)
session.add_all([p1,p2,p3])

In [31]:
session.commit()

2022-11-28 14:27:55,392 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-28 14:27:55,393 INFO sqlalchemy.engine.Engine INSERT INTO personas (ssn, firstname, lastname, gender, age) VALUES (?, ?, ?, ?, ?)
2022-11-28 14:27:55,393 INFO sqlalchemy.engine.Engine [generated in 0.00042s] ((8219821, 'Jhon', 'Smith', 'M', 31), (209130938, 'Lucas', 'Row', 'M', 23), (99999, 'Claudia', 'López', 'F', 45))
2022-11-28 14:27:55,394 INFO sqlalchemy.engine.Engine COMMIT


In [32]:
    session.query(Persona).all()

2022-11-28 14:27:55,424 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-28 14:27:55,426 INFO sqlalchemy.engine.Engine SELECT personas.ssn AS personas_ssn, personas.firstname AS personas_firstname, personas.lastname AS personas_lastname, personas.gender AS personas_gender, personas.age AS personas_age 
FROM personas
2022-11-28 14:27:55,426 INFO sqlalchemy.engine.Engine [generated in 0.00045s] ()


[21878 Mike Smith (M, 35),
 99999 Claudia López (F, 45),
 8219821 Jhon Smith (M, 31),
 209130938 Lucas Row (M, 23)]

In [33]:
resultados = session.query(Persona)
print ("==")
for r in resultados:
    print (r)

==
2022-11-28 14:27:55,460 INFO sqlalchemy.engine.Engine SELECT personas.ssn AS personas_ssn, personas.firstname AS personas_firstname, personas.lastname AS personas_lastname, personas.gender AS personas_gender, personas.age AS personas_age 
FROM personas
2022-11-28 14:27:55,461 INFO sqlalchemy.engine.Engine [cached since 0.03541s ago] ()
21878 Mike Smith (M, 35)
99999 Claudia López (F, 45)
8219821 Jhon Smith (M, 31)
209130938 Lucas Row (M, 23)


In [34]:
nuevos = resultados.filter(Persona.age < 40).all()
print (nuevos)

2022-11-28 14:27:55,497 INFO sqlalchemy.engine.Engine SELECT personas.ssn AS personas_ssn, personas.firstname AS personas_firstname, personas.lastname AS personas_lastname, personas.gender AS personas_gender, personas.age AS personas_age 
FROM personas 
WHERE personas.age < ?
2022-11-28 14:27:55,498 INFO sqlalchemy.engine.Engine [generated in 0.00077s] (40,)
[21878 Mike Smith (M, 35), 8219821 Jhon Smith (M, 31), 209130938 Lucas Row (M, 23)]


# Relaciones

In [35]:
%reset -f

In [36]:
from sqlalchemy import create_engine, ForeignKey, Column, String, Integer, CHAR
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

Base = declarative_base()

class Persona(Base):
    __tablename__ = "personas"
    
    ssn = Column(Integer, primary_key=True)
    firstname = Column(String)
    lastname = Column(String)
    gender = Column(CHAR)
    age = Column(Integer)
    
    def __init__(self, ssn, firstname, lastname, gender, age):
        self.ssn = ssn
        self.firstname = firstname
        self.lastname = lastname
        self.gender = gender.upper()
        self.age = age
        
    def __repr__(self):
        return f"{self.ssn} {self.firstname} {self.lastname} ({self.gender}, {self.age})"
    
class Cosa(Base):
    __tablename__ = "cosas"
    
    tid = Column(Integer, primary_key=True)
    description = Column(String)
    owner = Column(Integer, ForeignKey("personas.ssn"))
    
    def __repr__(self):
        return f"{self.tid} {self.description} owned by {self.owner}"
    
engine = create_engine("sqlite:///:memory:", echo=True)
Base.metadata.create_all(bind=engine)
Session = sessionmaker(bind=engine)
session = Session()


2022-11-28 14:27:55,634 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-28 14:27:55,635 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("personas")
2022-11-28 14:27:55,635 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-11-28 14:27:55,636 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("personas")
2022-11-28 14:27:55,636 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-11-28 14:27:55,637 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("cosas")
2022-11-28 14:27:55,637 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-11-28 14:27:55,638 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("cosas")
2022-11-28 14:27:55,638 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-11-28 14:27:55,640 INFO sqlalchemy.engine.Engine 
CREATE TABLE personas (
	ssn INTEGER NOT NULL, 
	firstname VARCHAR, 
	lastname VARCHAR, 
	gender CHAR, 
	age INTEGER, 
	PRIMARY KEY (ssn)
)


2022-11-28 14:27:55,640 INFO sqlalchemy.engine.Engine [no key 0.00047s] ()
2022-11-28 14:27:55,642 INFO sqlalchemy

In [37]:
p0 = Persona(21878, "Mike", "Smith", "m", 35)
p1 = Persona(8219821, "Jhon", "Smith", "m", 31)
p2 = Persona(209130938, "Lucas", "Row", "m", 23)
p3 = Persona(99999, "Claudia", "López", "f", 45)
session.add_all([p0, p1,p2,p3])
session.commit()

2022-11-28 14:27:55,693 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-28 14:27:55,695 INFO sqlalchemy.engine.Engine INSERT INTO personas (ssn, firstname, lastname, gender, age) VALUES (?, ?, ?, ?, ?)
2022-11-28 14:27:55,695 INFO sqlalchemy.engine.Engine [generated in 0.00052s] ((21878, 'Mike', 'Smith', 'M', 35), (8219821, 'Jhon', 'Smith', 'M', 31), (209130938, 'Lucas', 'Row', 'M', 23), (99999, 'Claudia', 'López', 'F', 45))
2022-11-28 14:27:55,696 INFO sqlalchemy.engine.Engine COMMIT


In [38]:
t1 = Cosa(tid= 23893232, description = "Una cosa", owner = p0.ssn)

2022-11-28 14:27:55,795 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-28 14:27:55,798 INFO sqlalchemy.engine.Engine SELECT personas.ssn AS personas_ssn, personas.firstname AS personas_firstname, personas.lastname AS personas_lastname, personas.gender AS personas_gender, personas.age AS personas_age 
FROM personas 
WHERE personas.ssn = ?
2022-11-28 14:27:55,800 INFO sqlalchemy.engine.Engine [generated in 0.00170s] (21878,)


In [39]:
session.add(t1)
session.commit()

2022-11-28 14:27:55,881 INFO sqlalchemy.engine.Engine INSERT INTO cosas (tid, description, owner) VALUES (?, ?, ?)
2022-11-28 14:27:55,882 INFO sqlalchemy.engine.Engine [generated in 0.00085s] (23893232, 'Una cosa', 21878)
2022-11-28 14:27:55,883 INFO sqlalchemy.engine.Engine COMMIT


In [40]:
t2 = Cosa(description = "PS5", owner = p1.ssn)
t3 = Cosa(description = "Auto", owner = p0.ssn)
session.add_all([t2,t3])
session.commit()

2022-11-28 14:29:24,195 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-28 14:29:24,197 INFO sqlalchemy.engine.Engine SELECT personas.ssn AS personas_ssn, personas.firstname AS personas_firstname, personas.lastname AS personas_lastname, personas.gender AS personas_gender, personas.age AS personas_age 
FROM personas 
WHERE personas.ssn = ?
2022-11-28 14:29:24,198 INFO sqlalchemy.engine.Engine [cached since 88.4s ago] (8219821,)
2022-11-28 14:29:24,201 INFO sqlalchemy.engine.Engine SELECT personas.ssn AS personas_ssn, personas.firstname AS personas_firstname, personas.lastname AS personas_lastname, personas.gender AS personas_gender, personas.age AS personas_age 
FROM personas 
WHERE personas.ssn = ?
2022-11-28 14:29:24,202 INFO sqlalchemy.engine.Engine [cached since 88.41s ago] (21878,)
2022-11-28 14:29:24,204 INFO sqlalchemy.engine.Engine INSERT INTO cosas (description, owner) VALUES (?, ?)
2022-11-28 14:29:24,205 INFO sqlalchemy.engine.Engine [generated in 0.00086s] ('PS5', 821

In [41]:
t3

2022-11-28 14:29:30,798 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-28 14:29:30,800 INFO sqlalchemy.engine.Engine SELECT cosas.tid AS cosas_tid, cosas.description AS cosas_description, cosas.owner AS cosas_owner 
FROM cosas 
WHERE cosas.tid = ?
2022-11-28 14:29:30,802 INFO sqlalchemy.engine.Engine [generated in 0.00135s] (23893234,)


23893234 Auto owned by 21878

In [42]:
results = session.query(Cosa, Persona).filter(Cosa.owner == Persona.ssn).filter(Persona.firstname == "Mike")

In [43]:
for r in results:
    print (r)

2022-11-28 14:30:50,301 INFO sqlalchemy.engine.Engine SELECT cosas.tid AS cosas_tid, cosas.description AS cosas_description, cosas.owner AS cosas_owner, personas.ssn AS personas_ssn, personas.firstname AS personas_firstname, personas.lastname AS personas_lastname, personas.gender AS personas_gender, personas.age AS personas_age 
FROM cosas, personas 
WHERE cosas.owner = personas.ssn AND personas.firstname = ?
2022-11-28 14:30:50,307 INFO sqlalchemy.engine.Engine [generated in 0.00575s] ('Mike',)
(23893232 Una cosa owned by 21878, 21878 Mike Smith (M, 35))
(23893234 Auto owned by 21878, 21878 Mike Smith (M, 35))
